In [7]:
import numpy as np
import regex as re
from pymongo import MongoClient
from pyeasyga import pyeasyga

Importing Pokemon Data:

In [8]:
client = MongoClient()
db = client['genetic_algo'] 

# Pokemon's teams collection
collection_poke_teams = db['pokemon_teams']
pokemon_teams = list(collection_poke_teams.find())

# Individual pokemon's collection
collec_indiv_pokemon = db['pokemons']
pokemons = list(collec_indiv_pokemon.find())


Defining some important functions:

In [23]:
gen_random_team()

[{'_id': ObjectId('66d60b4953343f07c4a35b0c'),
  'name': 'Riolu',
  'poke_id': '0446',
  'stamina': 120,
  'attack': 127,
  'defense': 78,
  'primaryType': 'Fighting',
  'primaryType_against_Bug': 0.5,
  'primaryType_against_Dark': 0.5,
  'primaryType_against_Dragon': 1,
  'primaryType_against_Electric': 1,
  'primaryType_against_Fairy': 2,
  'primaryType_against_Fighting': 0.5,
  'primaryType_against_Fire': 1,
  'primaryType_against_Flying': 2,
  'primaryType_against_Ghost': 1,
  'primaryType_against_Grass': 1,
  'primaryType_against_Ground': 1,
  'primaryType_against_Ice': 1,
  'primaryType_against_Normal': 1,
  'primaryType_against_Poison': 1,
  'primaryType_against_Psychic': 2,
  'primaryType_against_Rock': 0.5,
  'primaryType_against_Steel': 1,
  'primaryType_against_Water': 1,
  'CP': 993.0},
 {'_id': ObjectId('66d60b4953343f07c4a35995'),
  'name': 'Tentacool',
  'poke_id': '0071',
  'stamina': 120,
  'attack': 97,
  'defense': 149,
  'primaryType': 'Water',
  'primaryType_agains

In [26]:
def gen_random_team():

    client = MongoClient()
    db = client['genetic_algo'] 

    # Individual pokemon's collection
    collec_indiv_pokemon = db['pokemons']
    pokemons = list(collec_indiv_pokemon.find())
    
    
    # Random selector 
    rnd_selector = np.random.default_rng(seed = 1000090)

    poke_rival_tm_idx = rnd_selector.integers(low = 0, high = len(pokemons), size = 3)
    poke_rival_tm = [pokemons[poke_idx] for poke_idx in poke_rival_tm_idx]

    return poke_rival_tm


def best_against(pokeA, pokeB):
    
    best = 0
    pokemons_max_types = ['primaryType', 'secondaryType']
    
    for poke_typeA in pokemons_max_types:
        if poke_typeA in pokeA:

            for poke_typeB in pokemons_max_types:
                if poke_typeB in pokeB:
                    typeA_against_typeB = f'{poke_typeA}_against_{pokeB[poke_typeB]}'
                    if typeA_against_typeB in pokeA and pokeA[typeA_against_typeB] > best:

                        best = pokeA[typeA_against_typeB]
    return best
    

def battle(poke1, poke2):
    against1 = best_against(poke1, poke2)
    against2 = best_against(poke2, poke1)

    return poke1['CP'] * against1 - poke2['CP'] * against2

def fitness(individual, data):
    # Starts the fit value with 0
    fit = 0
    rival = gen_random_team()

    if individual.count(1) == 3:  # Check if 3 pokemons were selected
        selected_pokemon = [home_pokemon for selected, home_pokemon in zip(individual, data) if selected]
        
        # Cada Pokémon do time do indivíduo batalhará contra todos os 3 do time rival
        for home_pokemon in selected_pokemon:
            for rival_pokemon in rival:
                fit += battle(home_pokemon, rival_pokemon)
    
    return fit


### Training the GA

In [27]:
ga = pyeasyga.GeneticAlgorithm(
    pokemon_teams,  # Gera uma população inicial
    population_size=20,
    generations=50,
    crossover_probability=0.8,
    mutation_probability=0.2,
    elitism=True
)
ga.fitness_function = fitness
# Executar o algoritmo
ga.run()


In [28]:
print(ga.best_individual())

(0, [1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,